## Import libraries

In [1]:
import os
import json
import requests
import re

# from bs4 import BeautifulSoup

In [2]:
TOTAL_QUESTIONS_PARSED = 0
PARSING_RESULT_PATH = "questions_data/data/"

os.makedirs(PARSING_RESULT_PATH, exist_ok=True)

## 1. Data-Science-Interview-Questions-Answers
https://github.com/youssefHosni/Data-Science-Interview-Questions-Answers

In [3]:
# main_url = "https://raw.githubusercontent.com/youssefHosni/Data-Science-Interview-Questions-Answers/refs/heads/main/README.md"

# parsed from here already to files
# github_questions_urls = [
#     "https://raw.githubusercontent.com/youssefHosni/Data-Science-Interview-Questions-Answers/refs/heads/main/Machine%20Learning%20Interview%20Questions%20%26%20Answers%20for%20Data%20Scientists.md",
#     "https://raw.githubusercontent.com/youssefHosni/Data-Science-Interview-Questions-Answers/refs/heads/main/Deep%20Learning%20Questions%20%26%20Answers%20for%20Data%20Scientists.md",
#     "https://raw.githubusercontent.com/youssefHosni/Data-Science-Interview-Questions-Answers/refs/heads/main/Statistics%20Interview%20Questions%20%26%20Answers%20for%20Data%20Scientists.md",
#     "https://raw.githubusercontent.com/youssefHosni/Data-Science-Interview-Questions-Answers/refs/heads/main/Probability%20Interview%20Questions%20%26%20Answers%20for%20Data%20Scientists.md",
#     "https://raw.githubusercontent.com/youssefHosni/Data-Science-Interview-Questions-Answers/refs/heads/main/Python%20Interview%20Questions%20%26%20Answers%20for%20Data%20Scientists.md",
#     "https://raw.githubusercontent.com/youssefHosni/Data-Science-Interview-Questions-Answers/refs/heads/main/SQL%20%26%20DB%20Interview%20Questions%20%26%20Answers%20for%20Data%20Scientists.md",
#     "https://raw.githubusercontent.com/youssefHosni/Data-Science-Interview-Questions-Answers/refs/heads/main/Resume%20Based%20Questions.md"
# ]

medium_question_urls = [
    "https://levelup.gitconnected.com/top-large-language-models-llms-interview-questions-answers-d7b83f94c4e",
    "https://levelup.gitconnected.com/top-computer-vision-interview-questions-answers-part-1-7eddf45cfdf7",
    "https://levelup.gitconnected.com/top-computer-vision-interview-questions-answers-part-2-107244fc4289",
    "https://levelup.gitconnected.com/top-computer-vision-interview-questions-answers-part-3-1e43909131b2"
]

In [4]:
github_mds_path = "./questions_data/data-science-interview-questions-mds/"
github_mds_files = os.listdir(github_mds_path)

def parse_questions_answers(content: str) -> list[dict[str, str]]:
    # Regular expression to identify question headers
    # Modified regex to handle both "? ###" and "?###" formats
    question_pattern = re.compile(r"### Q\d+: (.+)")
    answer_start_pattern = re.compile(r"Answer:\s*(.*)", re.IGNORECASE)
    
    # List to hold the parsed questions and answers
    qa_list = []
    
    current_question = None
    current_answer = []
    
    # Split the content into lines
    lines = content.splitlines()
    
    def clean_answer(answer_lines):
        """ Helper function to clean answer lines """
        return "\n".join(answer_lines).strip()
    
    # Loop through each line to extract questions and answers
    for line in lines:
        # Match questions
        question_match = question_pattern.match(line)
        
        if question_match:
            # If we encounter a new question, save the previous question-answer pair (if any)
            if current_question:
                qa_list.append({
                    "question": current_question,
                    "answer": clean_answer(current_answer) if current_answer else ""
                })
            
            # Store the new question and reset the answer collection
            current_question = question_match.group(1)
            current_answer = []  # Reset for the new question
    
        else:
            # Match and capture the answer when 'Answer:' keyword is found
            answer_match = answer_start_pattern.match(line)
            if answer_match:
                # Start the answer with any text found on the same line after 'Answer:'
                current_answer.append(answer_match.group(1))
            elif current_question and current_answer is not None and not question_pattern.match(line):
                # Accumulate answer lines (anything after 'Answer:' until next question)
                current_answer.append(line)

    # Add the last question-answer pair after the loop
    if current_question:
        qa_list.append({
            "question": current_question,
            "answer": clean_answer(current_answer) if current_answer else ""
        })
    return qa_list


In [5]:
for file_name in github_mds_files:
    print(f"Now in {file_name}")
    concept_name = '_'.join(file_name.split('.')[0].split(' ')[:3]).lower()
    print(concept_name)

    with open(os.path.join(github_mds_path, file_name), 'r') as f:
        content = f.read()
        qa_list = parse_questions_answers(content)

    with open(f"questions_data/data/1_{concept_name}_parsed_questions.json", 'w') as f:
        json.dump(qa_list, f)

    print(f"Successfully parsed {len(qa_list)} questions and answers.")
    TOTAL_QUESTIONS_PARSED += len(qa_list)

    print()

print(f"Total questions parsed: {TOTAL_QUESTIONS_PARSED}")

Now in Machine Learning Interview Questions & Answers for Data Scientists.md
machine_learning_interview
Successfully parsed 36 questions and answers.

Now in Python Interview Questions & Answers for Data Scientists.md
python_interview_questions
Successfully parsed 14 questions and answers.

Now in Probability Interview Questions & Answers for Data Scientists.md
probability_interview_questions
Successfully parsed 17 questions and answers.

Now in Resume Based Questions.md
resume_based_questions
Successfully parsed 2 questions and answers.

Now in Deep Learning Questions & Answers for Data Scientists.md
deep_learning_questions
Successfully parsed 44 questions and answers.

Now in SQL & DB Interview Questions & Answers for Data Scientists.md
sql_&_db
Successfully parsed 12 questions and answers.

Now in Statistics Interview Questions & Answers for Data Scientists.md
statistics_interview_questions
Successfully parsed 17 questions and answers.

Total questions parsed: 142


## 2. Data-Science-Interviews

https://github.com/alexeygrigorev/data-science-interviews/tree/master

In [6]:
github_mds_path = "./questions_data/data-science-interviews-mds/"
github_mds_files = os.listdir(github_mds_path)

github_mds_files

['theory.md']

P.S. Решил дропнуть technical.md, так как там в основном SQL, алгоритмы и просто Python, пока обойдемся без таких практических вопросов и сконцентрируемся на ML

In [7]:
def parse_questions_answers(content: str) -> list[dict[str, str]]:
    # Regular expression to identify question headers
    question_pattern = re.compile(r"\*\*(.+?)\*\*")

    # Regular expression to identify section headers (lines that start with "## ")
    section_header_pattern = re.compile(r"^##\s+.+")

    # List to hold the parsed questions and answers
    qa_list = []

    current_question = None
    current_answer = []

    # Split the content into lines
    lines = content.splitlines()

    def clean_answer(answer_lines):
        """ Helper function to clean answer lines """
        return "\n".join(answer_lines).strip()

    # Loop through each line to extract questions and answers
    for line in lines:
        # Match questions (lines with "**" on both sides)
        question_match = question_pattern.match(line)

        if question_match:
            # If we encounter a new question, save the previous question-answer pair (if any)
            if current_question:
                qa_list.append({
                    "question": current_question,
                    "answer": clean_answer(current_answer) if current_answer else ""
                })

            # Store the new question and reset the answer collection
            current_question = question_match.group(1).strip()  # Clean up any leading/trailing spaces
            current_answer = []  # Reset for the new question

        # Skip section headers
        elif section_header_pattern.match(line):
            # If it's a section header, we skip it and do not add it to the answer
            continue

        else:
            # Accumulate answer lines (anything after the question)
            current_answer.append(line)

    # Add the last question-answer pair after the loop
    if current_question:
        qa_list.append({
            "question": current_question,
            "answer": clean_answer(current_answer) if current_answer else ""
        })

    return qa_list


In [8]:
for file_name in github_mds_files:
    print(f"Now in {file_name}")
    concept_name = file_name.split('.')[0]
    print(concept_name)

    with open(os.path.join(github_mds_path, file_name), 'r') as f:
        content = f.read()
        qa_list = parse_questions_answers(content)

    with open(f"questions_data/data/2_{concept_name}_parsed_questions.json", 'w') as f:
        json.dump(qa_list, f)

    print(f"Successfully parsed {len(qa_list)} questions and answers.")
    TOTAL_QUESTIONS_PARSED += len(qa_list)

    print()

print(f"Total questions parsed: {TOTAL_QUESTIONS_PARSED}")

Now in theory.md
theory
Successfully parsed 166 questions and answers.

Total questions parsed: 308


## 3. Data-Science-Interview-Questions-And-Answers

https://github.com/iamtodor/data-science-interview-questions-and-answers/tree/master

In [15]:
github_mds_path = "./questions_data/data-science-interview-questions-and-answers-mds/"
github_mds_files = os.listdir(github_mds_path)

github_mds_files

['main.md']

In [18]:
import re

def parse_questions_answers(content: str) -> list[dict[str, str]]:
    # Regular expression to identify question headers (e.g., "## 1. Why do you use feature selection?")
    question_pattern = re.compile(r"^##\s*\d+\.\s*(.+)")
    
    # Regular expression to identify sub-section headers within the answers (e.g., "#### Filter Methods")
    subsection_pattern = re.compile(r"^####\s*(.+)")
    
    # List to hold the parsed questions and answers
    qa_list = []
    
    current_question = None
    current_answer = []
    
    # Split the content into lines
    lines = content.splitlines()
    
    def clean_answer(answer_lines):
        """ Helper function to clean answer lines """
        return "\n".join(answer_lines).strip()
    
    # Loop through each line to extract questions and answers
    for line in lines:
        # Match questions (lines that start with "## <number>.")
        question_match = question_pattern.match(line.strip())

        if question_match:
            # If we encounter a new question, save the previous question-answer pair (if any)
            if current_question:
                qa_list.append({
                    "question": current_question.strip(),
                    "answer": clean_answer(current_answer) if current_answer else ""
                })
            
            # Store the new question and reset the answer collection
            current_question = question_match.group(1).strip()  # Extract the question text
            current_answer = []  # Reset for the new question
        
        # Match subsection headers (lines that start with "####"), these belong to the answer
        elif subsection_pattern.match(line.strip()):
            # Add subsections as part of the answer (e.g., "#### Filter Methods")
            current_answer.append(f"\n\n####{subsection_pattern.match(line.strip()).group(1)}\n\n")
        
        else:
            # Accumulate answer lines (anything after the question that isn't a new question header)
            current_answer.append(line)

    # Add the last question-answer pair after the loop
    if current_question:
        qa_list.append({
            "question": current_question,
            "answer": clean_answer(current_answer) if current_answer else ""
        })
    
    return qa_list

In [19]:
TOTAL_QUESTIONS_PARSED = 308

file_name = github_mds_files[0]
print(f"Now in {file_name}")
concept_name = file_name.split('.')[0]
print(concept_name)

with open(os.path.join(github_mds_path, file_name), 'r') as f:
    content = f.read()
    qa_list = parse_questions_answers(content)

with open(f"questions_data/data/3_{concept_name}_parsed_questions.json", 'w') as f:
    json.dump(qa_list, f)

print(f"Successfully parsed {len(qa_list)} questions and answers.")
TOTAL_QUESTIONS_PARSED += len(qa_list)

print()

print(f"Total questions parsed: {TOTAL_QUESTIONS_PARSED}")

Now in main.md
main
Successfully parsed 33 questions and answers.

Total questions parsed: 341
